# RAILS — Überwachtes CNN-Training für Schienenuntergrund-Klassifikation

**Ziel:** Training und Evaluation eines überwachten CNN (Transfer Learning mit ResNet50V2) zur präzisen Klassifikation von Eisenbahn-Gleisbettbildern. Aufbauend auf der unüberwachten Clustering-Analyse implementiert dieses Notebook eine **vollständige, eigenständige ML-Pipeline**:

1. **Daten-I/O** aus TFRecord-Format (Parsing, Dekodierung, Normalisierung)
2. **Modelldefinition** (vortrainiertes ResNet50V2-Backbone + leichter Klassifikationskopf)
3. **Training** mit der _besten Konfiguration_ (siehe unten)
4. **Evaluation** mit detaillierten Metriken (Confusion Matrix, klassenweise Precision/Recall/F1, Macro/Weighted Scores)
5. **Visualisierung** (Trainingskurven, Confusion Matrix, klassenweise Balkendiagramme, Klassenverteilung)
6. **Artefakte** (gespeichertes Modell + CSV/JSON-Ergebnisse)

Das Notebook basiert auf den Erkenntnissen der vorhergehenden Clustering-Phase und nutzt die beste, durch systematische Hyperparameter-Optimierung ermittelte Konfiguration für das finale Training.

## Schnellstart & Konfiguration

**Was:**  
- Setzt die Pfade zu den **TFRecord**-Dateien für Training und Evaluation.  
- Definiert **Ausgabeverzeichnis** und **Modellnamen** für Trainingsartefakte.  
- Legt **Klassenliste** und **Bildabmessungen** (224×224×3) fest, passend zum verwendeten Backbone.  
- Bündelt die feste **Best-Configuration** (Lernrate, Batchgröße, Epochen, Val-Anteil, Early-Stopping-Patience).

**Warum:**  
- Zentrale Konfiguration ermöglicht **reproduzierbare** Läufe ohne Parameter-Sweeps.  
- TFRecords als einheitliches Eingabeformat erleichtern **schnelles Laden** via `tf.data`.  
- Feste Hyperparameter sichern **Vergleichbarkeit** und verhindern „Experiment Creep“.  
- Ein Val-Anteil und Early Stopping unterstützen **stabiles Trainieren** und vermeiden Über-/Unteranpassung.


In [ ]:
# TFRecord inputs
TRAIN_TFRECORD_PATH = "/media/andi/ssd2/dev/code/Overseer2/data/inputs/MultiLabel_TB_small_08-25.tfrecord"   # <-- EDIT
EVAL_TFRECORD_PATH  = "/media/andi/ssd2/dev/code/Overseer2/data/inputs/MultiLabel_TB_Evaluation_08-25.tfrecord"    # <-- EDIT

# Output directory + model name
OUTPUT_DIR = "./outputs_trackbed"
MODEL_NAME = "PT_MultiClassResNet50_Trackbed"

# Label mapping and image shape (matches our ResNet50V2 setup)
CLASSES = ["ASPHALT", "BALLAST", "GRAS", "STONE", "ERROR"]
NUM_CLASSES = len(CLASSES)
IMG_HEIGHT = 224
IMG_WIDTH  = 224
IMG_DEPTH  = 3

# "Best" training configuration (as agreed/established externally)
BEST_CONFIG = {
    "learning_rate": 1e-4,
    "batch_size": 32,
    "num_epochs": 30,
    # You can tweak these without changing the core "best" idea
    "val_fraction": 0.2,         # reserve a small fraction of TRAIN for validation
    "early_stopping_patience": 5 # be gentle to avoid over/under-fitting
}


## 1. Setup: Umgebung & Reproduzierbarkeit

**Was:**
- Initialisierung von TensorFlow und Import der erforderlichen Bibliotheken
- Setzen von Seeds für Determinismus (soweit auf GPU praktikabel)
- Aktivierung des GPU-Memory-Growth zur Vermeidung von OOM-Fehlern

**Warum:**
- **Reproduzierbarkeit:** Konsistente Ergebnisse über mehrere Trainingsläufe hinweg
- **GPU-Optimierung:** Effiziente Speichernutzung verhindert Speicherüberläufe
- **Zentralisierte Konfiguration:** Alle Seeds und Umgebungseinstellungen an einem Ort

**Outputs:**
- Konfigurierte TensorFlow-Umgebung mit aktiviertem Memory-Growth
- Erstellung des Output-Verzeichnisses für Modell-Artefakte
- Ausgabe der TensorFlow-Version und des absoluten Output-Pfads

In [ ]:

import os, json, random, math, itertools
from pathlib import Path
from datetime import datetime

import numpy as np
import tensorflow as tf

# Reproducibility (best-effort on GPU)
SEED = 123
os.environ["PYTHONHASHSEED"] = str(SEED)
tf.random.set_seed(SEED)
np.random.seed(SEED)
random.seed(SEED)

# GPU memory growth (optional but recommended)
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print(f"Enabled memory growth for {len(gpus)} GPU(s).")
    except Exception as e:
        print(f"Could not set memory growth: {e}")

# Create output dir
Path(OUTPUT_DIR).mkdir(parents=True, exist_ok=True)

print("TF version:", tf.__version__)
print("Output dir:", os.path.abspath(OUTPUT_DIR))


## 2. Daten-Pipeline: TFRecord → `tf.data.Dataset`

**Was:**
- Implementierung der Daten-Pipeline mit dem Schema aus `create_trackbed_tfrecord.ipynb`
- Mehrstufige Verarbeitung: Parsing, Dekodierung, Normalisierung und Batching

**Verarbeitungsschritte:**

1. **Feature-Parsing** aus jedem TFRecord-Beispiel
2. **Dekodierung** Raw Bytes → Bildtensor; Sicherstellung 3 Kanäle; **Resize auf 224×224**
3. **One-Hot-Encoding** der Labels (5 Klassen)
4. **Normalisierung** auf `[0,1]`-Bereich
5. **Shuffle + Split** von `TRAIN` in **Train/Validation** (nach `val_fraction`)
   Der `EVAL` TFRecord wird unverändert für die finale Evaluation geladen

**Warum:**
- **Konsistenz:** Identisches Schema wie bei der TFRecord-Erstellung gewährleistet fehlerfreie Datenverarbeitung
- **Effizienz:** `tf.data.AUTOTUNE` und Prefetching optimieren die Datenlade-Performance
- **Flexibilität:** Separate Funktionen für Training (mit Shuffling) und Evaluation (deterministisch)

**Besonderheiten:**
- Automatische Grayscale→RGB-Konvertierung für Kompatibilität mit vortrainierten Modellen
- Deterministische Train/Val-Aufteilung durch `take/skip` (reproduzierbar bei unveränderter Dateireihenfolge)
- Rückgabe von Dateinamen bei Evaluation für detaillierte Fehleranalyse

In [ ]:

# Feature schema (must match TFRecord writer)
FEATURE_DESC = {
    'image_filename': tf.io.FixedLenFeature([], tf.string),
    'image_raw':      tf.io.FixedLenFeature([], tf.string),
    'height':         tf.io.FixedLenFeature([], tf.int64),
    'width':          tf.io.FixedLenFeature([], tf.int64),
    'depth':          tf.io.FixedLenFeature([], tf.int64),
    'label':          tf.io.FixedLenFeature([], tf.int64),
    'class_name':     tf.io.FixedLenFeature([], tf.string),
}

def _parse_tfrecord(proto):
    """Parse a single Example proto."""
    return tf.io.parse_single_example(proto, FEATURE_DESC)

def _decode_and_preprocess(feat_dict):
    """Decode bytes → image; enforce 3 channels; resize to 224x224; one‑hot label."""
    img = tf.io.decode_raw(feat_dict['image_raw'], tf.uint8)
    h   = tf.cast(feat_dict['height'], tf.int32)
    w   = tf.cast(feat_dict['width'],  tf.int32)
    d   = tf.cast(feat_dict['depth'],  tf.int32)
    img = tf.reshape(img, [h, w, d])

    # If single-channel, convert to RGB for pretrained models
    def to_rgb(x):
        return tf.image.grayscale_to_rgb(x)

    img = tf.cond(tf.equal(d, 1), lambda: to_rgb(img), lambda: img)

    # Resize to model input
    img = tf.image.resize(img, [IMG_HEIGHT, IMG_WIDTH])
    img = tf.cast(img, tf.float32) / 255.0  # normalize

    label_index = tf.cast(feat_dict['label'], tf.int32)
    label_1h    = tf.one_hot(label_index, depth=NUM_CLASSES)
    return img, label_1h

def _decode_with_filename(feat_dict):
    """Variant that also returns the original filename for evaluation/analysis."""
    img, label_1h = _decode_and_preprocess(feat_dict)
    return img, label_1h, feat_dict['image_filename']

def _count_records(tfrecord_path):
    """Count number of examples in a single-file TFRecord."""
    return sum(1 for _ in tf.data.TFRecordDataset(tfrecord_path))

def load_train_val_ds(tfrecord_path, batch_size, val_fraction=0.1, shuffle_multiplier=20):
    """Create train/val datasets from a single TFRecord file by a deterministic split."""
    n_total = _count_records(tfrecord_path)
    n_val   = max(1, int(round(n_total * float(val_fraction))))
    n_train = max(1, n_total - n_val)
    print(f"Found {n_total} samples → train: {n_train}, val: {n_val}")

    raw = tf.data.TFRecordDataset(tfrecord_path)
    raw = raw.map(_parse_tfrecord, num_parallel_calls=tf.data.AUTOTUNE)

    # We perform a simple split by 'take/skip' (repeatable as long as the file order doesn't change).
    # For stronger randomness across epochs, you could shuffle before splitting,
    # but then report the exact split seed in your paper.
    train_raw = raw.take(n_train)
    val_raw   = raw.skip(n_train)

    # Build train ds
    train_ds = (train_raw
                .shuffle(buffer_size=batch_size*shuffle_multiplier, seed=SEED, reshuffle_each_iteration=True)
                .map(_decode_and_preprocess, num_parallel_calls=tf.data.AUTOTUNE)
                .batch(batch_size)
                .prefetch(tf.data.AUTOTUNE))

    # Build val ds
    val_ds = (val_raw
              .map(_decode_and_preprocess, num_parallel_calls=tf.data.AUTOTUNE)
              .batch(batch_size)
              .prefetch(tf.data.AUTOTUNE))

    return train_ds, val_ds, n_train, n_val

def load_eval_ds(tfrecord_path, batch_size):
    raw = tf.data.TFRecordDataset(tfrecord_path)
    raw = raw.map(_parse_tfrecord, num_parallel_calls=tf.data.AUTOTUNE)
    ds  = (raw
           .map(_decode_with_filename, num_parallel_calls=tf.data.AUTOTUNE)
           .batch(batch_size)
           .prefetch(tf.data.AUTOTUNE))
    return ds


## 3. Modellarchitektur: ResNet50V2 + Klassifikationskopf

**Was:**
- Wiederverwendung der bewährten Architektur aus vorhergehenden Experimenten (vgl. [Report](https://wandb.ai/a-komrowski-nordakademie/multilabel-trackbed/reports/MultiClass-Trackbed-Detection--VmlldzoxNDE5Mjk2Nw?accessToken=ey801pz851tmsag3z39y5d10rkljuxcnzq4bcal3lwc0xiaog6qfj2o2i6r9de1n))
- **ResNet50V2** (ImageNet vortrainiert, eingefroren) + leichter MLP-Klassifikationskopf
- Loss-Funktion: **Categorical Cross-Entropy** (Single-Label, 5-Wege-Softmax)

**Warum:**
- **Transfer Learning:** Vortrainierte Features von ImageNet bieten robuste Basis für Bildklassifikation
- **Eingefrorenes Backbone:** Verhindert Overfitting bei begrenzten Trainingsdaten und reduziert Trainingszeit
- **Kategoriale Accuracy:** Passende Metrik für Multi-Class Single-Label-Klassifikation

**Architektur-Details:**
- **Input:** 224×224×3 (RGB-Bilder)
- **Backbone:** ResNet50V2 mit Global Average Pooling
- **Classifier:** Flatten → Dropout(0.3) → Dense(64) → Dense(32) → Dense(16) → Dense(5, softmax)
- **Optimizer:** Adam mit exponentieller Learning Rate Decay (initial_lr=1e-4, decay_rate=0.9)

**Outputs:**
- Kompiliertes Keras-Modell mit konfigurierten Optimizer und Metriken
- Modell-Summary zur Architektur-Verifikation

In [ ]:

from tensorflow.keras import Model, Input
from tensorflow.keras.layers import Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import ResNet50V2

def build_pt_multilabel_resnets_trackbed(initial_lr=1e-4, loss_fn='categorical_crossentropy'):
    # Input
    inp = Input(shape=(IMG_HEIGHT, IMG_WIDTH, IMG_DEPTH))

    # Pretrained backbone (frozen)
    base = ResNet50V2(include_top=False, weights='imagenet', input_shape=(IMG_HEIGHT, IMG_WIDTH, IMG_DEPTH), pooling='avg')
    base.trainable = False

    x = base(inp)
    x = Flatten()(x)
    x = Dropout(0.3)(x)
    x = Dense(64, activation='relu')(x)
    x = Dense(32, activation='relu')(x)
    x = Dense(16, activation='relu')(x)
    out = Dense(NUM_CLASSES, activation='softmax')(x)

    model = Model(inputs=inp, outputs=out)

    # Exponential decay on LR (as in our reference)
    lr_sched = tf.keras.optimizers.schedules.ExponentialDecay(
        initial_learning_rate=float(initial_lr),
        decay_steps=10_000,
        decay_rate=0.9
    )

    opt = Adam(learning_rate=lr_sched)
    model.compile(optimizer=opt, loss=loss_fn, metrics=[tf.keras.metrics.CategoricalAccuracy(name='categorical_accuracy')])
    return model

model = build_pt_multilabel_resnets_trackbed(initial_lr=BEST_CONFIG["learning_rate"], loss_fn='categorical_crossentropy')
model.summary()


## 4. Training: Beste Konfiguration (Single Run)

**Was:**
- Training **ausschließlich** mit der zuvor als optimal identifizierten Konfiguration:
  `learning_rate=1e-4`, `batch_size=32`, `num_epochs=30`
- Reservation von `val_fraction` (Standard 20%) des Trainingsdatensatzes für Validation

**Warum:**
- **Fokussierte Analyse:** Konzentration auf die beste Konfiguration statt erneuter Hyperparameter-Suche
- **Effizienz:** Direktes Training mit bewährten Parametern spart Rechenzeit
- **Vergleichbarkeit:** Konsistente Basis für Evaluation und Dokumentation

**Callbacks & Monitoring:**
- **ModelCheckpoint:** Speichert bestes Modell basierend auf `val_loss`  
  - *Warum sinnvoll?*  
    - `val_loss` ist die optimierte Zielgröße und reagiert sensitiver als Accuracy.  
    - `save_best_only=True` verhindert Modell-Wildwuchs und hält genau **ein** bestes Artefakt bereit.  
    - Garantiert ein reproduzierbares Referenzmodell für Auswertung/Deployment.
- **EarlyStopping:** Verhindert Overfitting mit konfigurierbarer Patience (Standard: 6 Epochen)  
  - *Warum sinnvoll?*  
    - Bricht stagnierende Läufe früh ab → spart Rechenzeit/Ressourcen.  
    - `restore_best_weights=True` stellt direkt den besten Gewichtsstand für die Auswertung bereit.
- **CSVLogger:** Persistiert Trainingsmetriken für spätere Analyse

**Outputs:**
- Trainiertes Modell gespeichert als `.keras`-Datei
- Training-Logs als CSV mit Verlauf von Loss und Accuracy
- Konfigurationsdateien (JSON) für Reproduzierbarkeit
- Konsolen-Ausgabe der finalen Pfade und Datensatzgrößen


In [ ]:

from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, CSVLogger

BATCH_SIZE = int(BEST_CONFIG["batch_size"])
EPOCHS     = int(BEST_CONFIG["num_epochs"])
VAL_FRAC   = float(BEST_CONFIG["val_fraction"])

# Load datasets
train_ds, val_ds, n_train, n_val = load_train_val_ds(TRAIN_TFRECORD_PATH, batch_size=BATCH_SIZE, val_fraction=VAL_FRAC)
eval_ds = load_eval_ds(EVAL_TFRECORD_PATH, batch_size=BATCH_SIZE)

# Callbacks & paths
timestamp   = datetime.now().strftime("%Y%m%d-%H%M%S")
run_dir     = Path(OUTPUT_DIR) / f"{MODEL_NAME}__{timestamp}"
run_dir.mkdir(parents=True, exist_ok=True)

MODEL_PATH  = str(run_dir / f"{MODEL_NAME}.keras")
LOG_CSV     = str(run_dir / "training_log.csv")
CFG_JSON    = str(run_dir / "config.json")
CLASSES_JSON= str(run_dir / "classes.json")

# Save config & classes for reproducibility
with open(CFG_JSON, "w") as f:
    json.dump(BEST_CONFIG, f, indent=2)
with open(CLASSES_JSON, "w") as f:
    json.dump(CLASSES, f, indent=2)

cbs = [
    ModelCheckpoint(MODEL_PATH, monitor='val_loss', save_best_only=True, save_weights_only=False, verbose=1),
    EarlyStopping(monitor='val_loss', patience=int(BEST_CONFIG["early_stopping_patience"]), restore_best_weights=True, verbose=1),
    CSVLogger(LOG_CSV) #eig raus
]

history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS,
    callbacks=cbs,
    verbose=1
)

print("\nBest model saved to:", MODEL_PATH)
print("Logs saved to:", LOG_CSV)
print("Train/Val sizes:", n_train, n_val)


## 5. Trainingskurven: Verlaufsanalyse

**Was:**
- Visualisierung von Loss und Categorical Accuracy über alle Epochen
- Separate Kurven für Training und Validation zur Overfitting-Erkennung

**Warum:**
- **Konvergenz-Analyse:** Überprüfung ob das Training erfolgreich konvergiert ist
- **Overfitting-Detektion:** Auseinanderlaufende Train/Val-Kurven zeigen Überanpassung
- **Model-Validierung:** Visueller Nachweis einer stabilen und erfolgreichen Optimierung

**Interpretation:**
- **Idealer Verlauf:** Beide Kurven sinken (Loss) bzw. steigen (Accuracy) und konvergieren
- **Overfitting:** Training-Metrik verbessert sich weiter, während Validation stagniert oder schlechter wird
- **Underfitting:** Beide Kurven haben noch nicht konvergiert und zeigen weiteres Verbesserungspotential

**Outputs:**
- Zwei Matplotlib-Plots: Loss-Verlauf und Accuracy-Verlauf
- Visuelle Dokumentation der Trainingsqualität

In [ ]:

import matplotlib.pyplot as plt

# Plot: Loss
plt.figure()
plt.plot(history.history.get('loss', []), label='train_loss')
plt.plot(history.history.get('val_loss', []), label='val_loss')
plt.title('Loss over epochs')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

# Plot: Categorical Accuracy
plt.figure()
plt.plot(history.history.get('categorical_accuracy', []), label='train_cat_acc')
plt.plot(history.history.get('val_categorical_accuracy', []), label='val_cat_acc')
plt.title('Categorical Accuracy over epochs')
plt.xlabel('Epoch')
plt.ylabel('Categorical Accuracy')
plt.legend()
plt.show()


## 6. Evaluation: Detaillierte Leistungsanalyse auf Hold-out-Datensatz

**Was:**
- Umfassende Evaluation des besten Modells auf dem separaten EVAL-Datensatz
- Berechnung aller relevanten Klassifikationsmetriken und detaillierte Fehleranalyse

**Berechnete Metriken:**
- **Confusion Matrix** (Gesamt-Übersicht der Klassifikationsergebnisse)
- **Pro Klasse:** Precision, Recall (Sensitivity), Specificity, F1-Score, Accuracy
- **Fehlerraten:** Type I/II Error Rates
- **Aggregate Metriken:** Macro (ungewichtet) und Weighted (gewichtet) Averages

**Gespeicherte Artefakte:**
- `evaluation_summary_*.csv` — Vollständige Metrik-Tabelle
- `confusion_matrix_*.csv` — Raw Counts der Confusion Matrix
- `false_inferences_*.json` — Dateinamen aller FP/FN + Fehlklassifikationen (mit Vorhersage vs. wahre Klasse)

**Warum:**
- **Objektive Bewertung:** Hold-out-Set war nie in Training/Validation involviert
- **Detaillierte Analyse:** Pro-Klassen-Metriken decken klassenspezifische Schwächen auf
- **Fehleranalyse:** Identifikation problematischer Bilder für weitere Untersuchungen
- **Reproduzierbarkeit:** Alle Metriken und Fehler werden persistent gespeichert

**Visualisierungen:**
- **Confusion Matrix Heatmap** mit Zahlenwerten
- **F1-Score Balkendiagramm** pro Klasse
- **Klassenverteilung** des Evaluationsdatensatzes
- **False Positives vs. False Negatives** Vergleich pro Klasse

**Outputs:**
- CSV/JSON-Dateien mit allen Metriken und Fehlern
- Vier Visualisierungen zur Ergebnisinterpretation
- Konsolen-Summary mit Kernmetriken (Overall Accuracy, Macro F1, etc.)

In [ ]:

from sklearn.metrics import confusion_matrix
import csv
import math
import numpy as np
import matplotlib.pyplot as plt

# Ensure we load the best-saved model from disk (even if EarlyStopping restored in-memory weights)
best_model = tf.keras.models.load_model(MODEL_PATH, compile=False)

# Predict across the eval set
all_true_idx = []
all_pred_idx = []
false_dict = {
    'false_positive': {c: [] for c in CLASSES},
    'false_negative': {c: [] for c in CLASSES},
    'misclassified': []
}

for batch_imgs, batch_trues, batch_fns in eval_ds:
    probs = best_model.predict(batch_imgs, verbose=0)
    pred_idx = np.argmax(probs, axis=1)
    true_idx = np.argmax(batch_trues.numpy(), axis=1)

    all_pred_idx.extend(list(pred_idx))
    all_true_idx.extend(list(true_idx))

    # Track errors with filenames + confidence
    for ti, pi, fn, p in zip(true_idx, pred_idx, batch_fns.numpy(), probs):
        filename = fn.decode('utf-8') if isinstance(fn, (bytes, bytearray)) else str(fn)
        conf = float(np.max(p))
        if ti != pi:
            false_dict['misclassified'].append({
                'filename': filename,
                'true_class': CLASSES[int(ti)],
                'predicted_class': CLASSES[int(pi)],
                'confidence': conf
            })
        # Per-class FP/FN views
        for ci in range(NUM_CLASSES):
            # Binary view for class ci
            true_bin = (ti == ci)
            pred_bin = (pi == ci)
            if (not true_bin) and pred_bin:
                false_dict['false_positive'][CLASSES[ci]].append(f"{filename}_{conf:.3f}")
            if true_bin and (not pred_bin):
                false_dict['false_negative'][CLASSES[ci]].append(f"{filename}_{conf:.3f}")

all_true_idx = np.array(all_true_idx, dtype=int)
all_pred_idx = np.array(all_pred_idx, dtype=int)

# Overall confusion matrix (NUM_CLASSES x NUM_CLASSES)
overall_conf = confusion_matrix(all_true_idx, all_pred_idx, labels=list(range(NUM_CLASSES)))

# Build per-class binary TP/FP/FN/TN
binary_conf = np.zeros((NUM_CLASSES, 4), dtype=float)  # TP, FP, FN, TN
for ci in range(NUM_CLASSES):
    # For class ci: positive if true==ci / predicted==ci
    tp = np.sum((all_true_idx == ci) & (all_pred_idx == ci))
    fp = np.sum((all_true_idx != ci) & (all_pred_idx == ci))
    fn = np.sum((all_true_idx == ci) & (all_pred_idx != ci))
    tn = np.sum((all_true_idx != ci) & (all_pred_idx != ci))
    binary_conf[ci] = [tp, fp, fn, tn]

def _safe_div(a, b):
    return (a / b) if b != 0 else 0.0

# Compute metrics
tps, fps, fns, tns = binary_conf[:,0], binary_conf[:,1], binary_conf[:,2], binary_conf[:,3]
accuracies   = np.array([_safe_div(tp+tn, tp+fp+fn+tn) for tp,fp,fn,tn in binary_conf])
recalls      = np.array([_safe_div(tp, tp+fn) for tp,fn in zip(tps,fns)])              # aka sensitivity
specificity  = np.array([_safe_div(tn, tn+fp) for tn,fp in zip(tns,fps)])
typeI_err    = np.array([_safe_div(fp, fp+tn) for fp,tn in zip(fps,tns)])
typeII_err   = np.array([_safe_div(fn, tp+fn) for tp,fn in zip(tps,fns)])
precisions   = np.array([_safe_div(tp, tp+fp) for tp,fp in zip(tps,fps)])
f1_scores    = np.array([_safe_div(2*p*r, p+r) for p,r in zip(precisions,recalls)])

overall_acc  = _safe_div(np.trace(overall_conf), np.sum(overall_conf))

macro_precision = float(np.mean(precisions)) if len(precisions) else 0.0
macro_recall    = float(np.mean(recalls))    if len(recalls)    else 0.0
macro_f1        = float(np.mean(f1_scores))  if len(f1_scores)  else 0.0

supports = tps + fns
total    = np.sum(supports) if np.sum(supports) > 0 else 1.0

metrics = {
    "Accuracy": list(map(float, accuracies)) + [float(overall_acc)],
    "Precision": list(map(float, precisions)) + [float(macro_precision)],
    "Recall": list(map(float, recalls)) + [float(macro_recall)],
    "Specificity": list(map(float, specificity)) + [float(np.mean(specificity) if len(specificity) else 0.0)],
    "F1-Score": list(map(float, f1_scores)) + [float(macro_f1)],
    "Type I Error": list(map(float, typeI_err)) + [float(np.mean(typeI_err) if len(typeI_err) else 0.0)],
    "Type II Error": list(map(float, typeII_err)) + [float(np.mean(typeII_err) if len(typeII_err) else 0.0)]
}

# Save CSV metrics and confusion matrix + false inferences JSON
eval_csv = str(Path(run_dir) / f"evaluation_summary_{MODEL_NAME}.csv")
conf_csv = str(Path(run_dir) / f"confusion_matrix_{MODEL_NAME}.csv")
false_json = str(Path(run_dir) / f"false_inferences_{MODEL_NAME}.json")

with open(eval_csv, 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(["Metric", "Class", "Value"])
    # per-class
    for name, vals in metrics.items():
        if name.startswith("Weighted"):
            continue
        for idx, v in enumerate(vals[:-1]):
            writer.writerow([name, CLASSES[idx], v])
    # macro/overall
    writer.writerow([])
    writer.writerow(["Metric", "Overall/Average", "Value"])
    for name, vals in metrics.items():
        if name.startswith("Weighted"):
            writer.writerow([name, "Weighted", vals[0]])
        else:
            writer.writerow([name, "Macro Average", vals[-1]])

np.savetxt(conf_csv, overall_conf, delimiter=',', fmt='%d', header=','.join(CLASSES), comments='')

with open(false_json, 'w') as jf:
    json.dump(false_dict, jf, indent=2)

print("Saved:", eval_csv)
print("Saved:", conf_csv)
print("Saved:", false_json)

# --- Visualizations -----------------------------------------------------------

# (1) Confusion matrix heatmap with better color scheme
plt.figure(figsize=(8, 6))
im = plt.imshow(overall_conf, cmap='Blues', interpolation='nearest')
plt.title('Confusion Matrix (counts)', fontsize=14, fontweight='bold')
plt.xticks(ticks=range(NUM_CLASSES), labels=CLASSES, rotation=45, ha='right')
plt.yticks(ticks=range(NUM_CLASSES), labels=CLASSES)
plt.xlabel('Predicted', fontsize=12)
plt.ylabel('True', fontsize=12)

# Add text annotations with good contrast
for i in range(NUM_CLASSES):
    for j in range(NUM_CLASSES):
        text_color = 'white' if overall_conf[i, j] > overall_conf.max() / 2 else 'black'
        plt.text(j, i, str(overall_conf[i, j]), ha='center', va='center', 
                color=text_color, fontsize=11, fontweight='bold')

plt.colorbar(im, shrink=0.8)
plt.tight_layout()
plt.show()

# (2) Comprehensive Metrics Bar Chart
fig, ax = plt.subplots(figsize=(14, 8))

# Prepare data for visualization
metric_names = ['Precision', 'Recall', 'Specificity', 'F1-Score']
x = np.arange(len(CLASSES))
width = 0.2

colors = ['#1f77b4', '#ff7f0e', '#d62728', '#2ca02c']

for i, metric in enumerate(metric_names):
    values = metrics[metric][:-1]  # Exclude macro average
    bars = ax.bar(x + i*width, values, width, label=metric, color=colors[i], alpha=0.8)
    
    # Add value labels on top of bars
    for j, (bar, val) in enumerate(zip(bars, values)):
        height = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2., height + 0.01,
                f'{val:.3f}', ha='center', va='bottom', fontsize=9, fontweight='bold')

ax.set_xlabel('Classes', fontsize=12, fontweight='bold')
ax.set_ylabel('Metric Values', fontsize=12, fontweight='bold')
ax.set_title('Per-Class Performance Metrics', fontsize=14, fontweight='bold')
ax.set_xticks(x + width * 1.5)
ax.set_xticklabels(CLASSES, rotation=30, ha='right')
ax.legend(loc='upper left', bbox_to_anchor=(1, 1))
ax.set_ylim(0, 1.1)
ax.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

# (3) Alternative Suggestion: Radar Chart for Macro Metrics
fig, ax = plt.subplots(figsize=(8, 8), subplot_kw=dict(projection='polar'))

# Macro metrics for radar chart
radar_metrics = ['Precision', 'Recall', 'F1-Score', 'Specificity']
radar_values = [metrics[m][-1] for m in radar_metrics]  # Get macro averages

# Number of variables
N = len(radar_metrics)

# Compute angle for each axis
angles = [n / float(N) * 2 * np.pi for n in range(N)]
angles += angles[:1]  # Complete the circle

# Add values
radar_values += radar_values[:1]  # Complete the circle

# Plot
ax.plot(angles, radar_values, 'o-', linewidth=2, label='Macro Average', color='#1f77b4')
ax.fill(angles, radar_values, alpha=0.25, color='#1f77b4')

# Add labels
ax.set_xticks(angles[:-1])
ax.set_xticklabels(radar_metrics, fontsize=11)
ax.set_ylim(0, 1)
ax.set_yticks([0.2, 0.4, 0.6, 0.8, 1.0])
ax.set_yticklabels(['0.2', '0.4', '0.6', '0.8', '1.0'], fontsize=9)
ax.grid(True)

# Add value labels
for angle, value, metric in zip(angles[:-1], radar_values[:-1], radar_metrics):
    ax.text(angle, value + 0.05, f'{value:.3f}', ha='center', va='center', 
            fontsize=10, fontweight='bold', 
            bbox=dict(boxstyle='round,pad=0.2', facecolor='white', alpha=0.8))

ax.set_title('Overall Model Performance\n(Macro Averages)', fontsize=14, fontweight='bold', pad=20)
plt.tight_layout()
plt.show()

# (4) False Positives vs False Negatives per class
fp_counts = [len(false_dict['false_positive'][c]) for c in CLASSES]
fn_counts = [len(false_dict['false_negative'][c]) for c in CLASSES]

plt.figure(figsize=(10, 6))
x = np.arange(NUM_CLASSES)
plt.bar(x - 0.2, fp_counts, width=0.4, label='False Positives', color='#ff7f0e', alpha=0.8)
plt.bar(x + 0.2, fn_counts, width=0.4, label='False Negatives', color='#d62728', alpha=0.8)
plt.xticks(x, CLASSES, rotation=30, ha='right')
plt.title('False Positives vs False Negatives per Class', fontsize=14, fontweight='bold')
plt.xlabel('Class', fontsize=12)
plt.ylabel('Count', fontsize=12)
plt.legend()
plt.grid(True, alpha=0.3, axis='y')
plt.tight_layout()
plt.show()

print("\n=== EVAL SUMMARY ===")
print(f"Overall accuracy: {overall_acc:.4f}")
print(f"Macro F1-Score:  {macro_f1:.4f}")
print(f"Total samples:    {int(np.sum(overall_conf))}")
print(f"Total misclass.:  {len(false_dict['misclassified'])}")


## 7. Qualitative Stichprobe: Vorhersage-Visualisierung

**Was:**
- Visualisierung einiger Vorhersagen mit wahren Labels und Konfidenzwerten
- Direkte Anzeige aus `eval_ds` Tensoren (nicht von Festplatte)

**Warum:**
- **Qualitative Kontrolle:** Visueller Eindruck der Modell-Performance auf echten Bildern
- **Fehlerverständnis:** Erkennung von Mustern bei Fehlklassifikationen
- **Konfidenz-Analyse:** Bewertung der Modell-Sicherheit bei verschiedenen Vorhersagen

**Interpretation:**
- **Korrekte Vorhersagen:** Hohe Konfidenz bei richtigen Klassifikationen ist wünschenswert
- **Fehlklassifikationen:** Niedrige Konfidenz kann auf schwierige/mehrdeutige Bilder hinweisen
- **Systematische Fehler:** Wiederholte Verwechslungen zwischen bestimmten Klassen

**Outputs:**
- Grid-Anzeige mit 6 Beispielbildern
- Je Bild: Vorhersage, Konfidenz, wahre Klasse

In [ ]:

import matplotlib.pyplot as plt

def show_eval_samples(ds, model, k=6):
    imgs_shown = 0
    for imgs, labels, fns in ds:
        probs = model.predict(imgs, verbose=0)
        pred_idx = np.argmax(probs, axis=1)
        true_idx = np.argmax(labels.numpy(), axis=1)

        b = imgs.shape[0]
        rows = int(math.ceil(min(k, b) / 3))
        cols = 3 if k >= 3 else min(k, b)

        plt.figure(figsize=(cols*3, rows*3))
        for i in range(min(k, b)):
            ax = plt.subplot(rows, cols, i+1)
            ax.imshow(imgs[i].numpy())
            pidx = int(pred_idx[i])
            tidx = int(true_idx[i])
            conf = float(np.max(probs[i]))
            ax.set_title(f"pred: {CLASSES[pidx]} ({conf:.2f})\ntrue: {CLASSES[tidx]}")
            ax.axis('off')
        plt.tight_layout()
        plt.show()

        imgs_shown += min(k, b)
        if imgs_shown >= k:
            break

# Uncomment to preview a few eval samples (set k as needed)
show_eval_samples(eval_ds, best_model, k=6)


## 8. Dokumentation & Reproduzierbarkeit

**Ergebnis:**
Dieses Notebook liefert ein vollständig trainiertes und evaluiertes CNN-Modell für die Schienenuntergrund-Klassifikation mit umfassender Dokumentation aller Parameter, Metriken und Ergebnisse. Die systematische Herangehensweise und detaillierte Evaluation bilden eine solide Grundlage für den praktischen Einsatz in der Gleisbett-Analyse.

**Gespeicherte Artefakte:**
- **Hyperparameter** und **Klassennamen** werden neben dem trainierten Modell im Run-Ordner gespeichert
- **Bestes Modell** (nach `val_loss`) wird im Keras-Format gespeichert: `<OUTPUT_DIR>/<MODEL_NAME>__<timestamp>/<MODEL_NAME>.keras`
- **Metriken und Confusion Matrix** sind als CSV-Dateien verfügbar
- **Fehlklassifikationen** werden als JSON gespeichert (mit Dateinamen und Konfidenzwerten)

**Nächste Schritte (optional):**
- **Fine-Tuning:** Einfrieren der oberen ResNet-Schichten aufheben (mit kleiner Learning Rate)
- **Klassenbalancierung:** Class-balanced Sampling / Focal Loss bei unbalancierten Datensätzen
- **Cross-Validation:** Integration verschiedener TFRecord-Shards (falls verfügbar)